In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Main

> Here, we will define the main functionalities of the package.

In [ ]:
#| default_exp main

In [ ]:
#| export
#| hide
import argparse
from thucy.configuration import *
from thucy.toolbox import *
from agents import trace
from thucy.agents import *
import asyncio

In [ ]:
#| export
#| hide

def handle_config_command(args):
    if args.config_action == 'set':
        config.set_env_var(args.key, args.value)
    elif args.config_action == 'show':
        config.show_config()
    else:
        print("Use: thucy config [set|show]")

In [ ]:
#| export
#| hide

async def handle_verify_command(args):

    print("Setting up connection to Google's toolbox...")
    genai_mcp.connect()
    genai_mcp.set_specific_toolset(args.toolset)
    print("Success!")

    user_query = UserQuery(query=args.query)

    print("Starting Verification. Might take a while...!")
    with trace(args.workflow) as t:
        result = await verify(user_query)
    print("Success!")
    print(f"\nThucy Verdict: {result.verdict}\n")
    
    trace_id = t.trace_id
    file_id = f"{args.workflow}-{trace_id}"

    append_to_file(f"{file_id}.txt", user_query, "userQuery")
    append_to_file(f"{file_id}.txt", result, "verificationAnswer")

    print(f"Saved full report to results/{file_id}.txt")

    genai_mcp.close()
    print("Exiting!")

In [ ]:
#| export

def main():
    parser = argparse.ArgumentParser(description='Thucy CLI')

    subparsers = parser.add_subparsers(dest='command', help='Commands')

    # ---------------------------------------------
    # thucy verify "<query>" --workflow ... --toolset ...
    # ---------------------------------------------

    verify_parser = subparsers.add_parser('verify', help='Run claim verification')
    verify_parser.add_argument('query', help='Natural language query to verify')
    verify_parser.add_argument('--workflow', default='Default-Workflow',
                              help="Workflow name for the traces on OpenAI. Will also be the filename (with the unique trace).")
    verify_parser.add_argument('--toolset', help="Toolset name (e.g., seattle). For example, if given `seattle`, then, in the `tools.yaml` we must provide the toolsets `seattle-sql` and `seattle-schema`.", required=True)

    # ---------------------------------------------
    # thucy config ...
    # ---------------------------------------------
    config_parser = subparsers.add_parser('config', help='Manage configuration')
    config_subparsers = config_parser.add_subparsers(dest='config_action')

    # thucy config set KEY VALUE
    set_parser = config_subparsers.add_parser('set', help='Set a config value')
    set_parser.add_argument('key', help='Config key (e.g., OPENAI_API_KEY)')
    set_parser.add_argument('value', help='Config value')
    # thucy config list
    config_subparsers.add_parser('show', help='List available keys')
    
    args = parser.parse_args()
    
    if args.command == 'config':
        handle_config_command(args)
    elif args.command == 'verify':
        asyncio.run(handle_verify_command(args))
    else:
        parser.print_help()

In [ ]:
#| export

try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

In [ ]:
#| export
import asyncio

In [ ]:
#| export
#| notest
if __name__ == '__main__' and not IN_NOTEBOOK:
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()